# Type of Apartment VS Type of Neighbourhood vs Ratings

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
from sklearn.linear_model import LinearRegression
sb.set() # set the default Seaborn style for graphics

from textblob import TextBlob

from sklearn.utils import resample


from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats
from sklearn.preprocessing import OneHotEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.tree import plot_tree



from collections import Counter
air_data = pd.read_csv("listings_june.csv")
air_data.head()


,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,49091,https://www.airbnb.com/rooms/49091,20210629180711,2021-06-30,COZICOMFORT LONG TERM STAY ROOM 2,<b>The space</b><br />This is Room No. 2.(avai...,NaN,https://a0.muscache.com/pictures/24485541/67b0...,266763,https://www.airbnb.com/users/show/266763,...,5.00,4.00,4.00,NaN,f,2,0,2,0,0.01
1,50646,https://www.airbnb.com/rooms/50646,20210629180711,2021-06-30,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,https://a0.muscache.com/pictures/36342984/670d...,227796,https://www.airbnb.com/users/show/227796,...,4.94,4.72,4.50,NaN,f,1,0,1,0,0.22
2,56334,https://www.airbnb.com/rooms/56334,20210629180711,2021-06-30,COZICOMFORT,<b>The space</b><br />This is Room no. 1. USD7...,NaN,https://a0.muscache.com/pictures/20783922/1966...,266763,https://www.airbnb.com/users/show/266763,...,5.00,4.17,4.67,NaN,f,2,0,2,0,0.16
3,71609,https://www.airbnb.com/rooms/71609,20210629180711,2021-06-30,Ensuite Room (Room 1 & 2) near EXPO,For 3 rooms.Book room 1&2 and room 4<br /><br ...,NaN,https://a0.muscache.com/pictures/24453191/3580...,367042,https://www.airbnb.com/users/show/367042,...,4.78,4.26,4.32,NaN,t,4,0,4,0,0.29
4,71896,https://www.airbnb.com/rooms/71896,20210629180711,2021-06-30,B&B Room 1 near Airport & EXPO,<b>The space</b><br />Vocational Stay Deluxe B...,NaN,https://a0.muscache.com/pictures/2440674/ac4f4...,367042,https://www.airbnb.com/users/show/367042,...,4.43,4.17,4.04,NaN,t,4,0,4,0,0.34


In [2]:
air_data["property_type"]=air_data['property_type'].str.replace('[', '').str.replace(']','').str.replace('"','')
air_data["property_type"] =air_data["property_type"].to_frame()
air_data["property_type"].head()


C:\Users\junwe\AppData\Local\Temp/ipykernel_20276/3366525147.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  air_data["property_type"]=air_data['property_type'].str.replace('[', '').str.replace(']','').str.replace('"','')


0    Private room in apartment
1    Private room in apartment
2    Private room in apartment
3        Private room in villa
4        Private room in house
Name: property_type, dtype: object

In [3]:
air_data["neighbourhood_cleansed"]=air_data['neighbourhood_cleansed'].str.replace('[', '').str.replace(']','').str.replace('"','')
air_data["neighbourhood_cleansed"] =air_data["neighbourhood_cleansed"].to_frame()
air_data["neighbourhood_cleansed"].head()


C:\Users\junwe\AppData\Local\Temp/ipykernel_20276/2373546424.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  air_data["neighbourhood_cleansed"]=air_data['neighbourhood_cleansed'].str.replace('[', '').str.replace(']','').str.replace('"','')


0      Woodlands
1    Bukit Timah
2      Woodlands
3       Tampines
4       Tampines
Name: neighbourhood_cleansed, dtype: object

In [4]:
print(np.unique(air_data[['neighbourhood_cleansed']].values))


['Ang Mo Kio' 'Bedok' 'Bishan' 'Bukit Batok' 'Bukit Merah' 'Bukit Panjang'
 'Bukit Timah' 'Central Water Catchment' 'Choa Chu Kang' 'Clementi'
 'Downtown Core' 'Geylang' 'Hougang' 'Jurong East' 'Jurong West' 'Kallang'
 'Mandai' 'Marine Parade' 'Museum' 'Newton' 'Novena' 'Orchard' 'Outram'
 'Pasir Ris' 'Pioneer' 'Punggol' 'Queenstown' 'River Valley' 'Rochor'
 'Sembawang' 'Sengkang' 'Serangoon' 'Singapore River' 'Southern Islands'
 'Sungei Kadut' 'Tampines' 'Tanglin' 'Toa Payoh' 'Tuas'
 'Western Water Catchment' 'Woodlands' 'Yishun']


In [5]:
print(np.unique(air_data[['property_type']].values))

['Boat' 'Campsite' 'Chambre privée dans\xa0: appartement'
 'Entire apartment' 'Entire bungalow' 'Entire chalet' 'Entire condominium'
 'Entire guest suite' 'Entire guesthouse' 'Entire home/apt' 'Entire house'
 'Entire loft' 'Entire place' 'Entire serviced apartment'
 'Entire townhouse' 'Entire villa' 'Private room'
 'Private room in apartment' 'Private room in bed and breakfast'
 'Private room in boat' 'Private room in bungalow'
 'Private room in casa particular' 'Private room in chalet'
 'Private room in condominium' 'Private room in earth house'
 'Private room in guest suite' 'Private room in guesthouse'
 'Private room in hostel' 'Private room in house'
 'Private room in lighthouse' 'Private room in loft'
 'Private room in serviced apartment' 'Private room in tent'
 'Private room in townhouse' 'Private room in villa' 'Room in aparthotel'
 'Room in apartment' 'Room in bed and breakfast' 'Room in boutique hotel'
 'Room in heritage hotel' 'Room in hostel' 'Room in hotel'
 'Room in servic

In [6]:
air_data["num_of_amenities"]= air_data["amenities"].apply(lambda n: len(n.split(",")))
air_data["num_of_amenities"].head()

0     7
1    12
2     8
3    25
4    21
Name: num_of_amenities, dtype: int64

In [7]:
air_data["num_of_veri"]= air_data["host_verifications"].apply(lambda n: len(n.split(',')))
air_data["num_of_veri"].head()

0    9
1    8
2    9
3    5
4    5
Name: num_of_veri, dtype: int64

In [8]:
air_data["price"]=  air_data['price'].str.replace(',', '').str.replace('$', '')
air_data["price"] = pd.to_numeric(air_data["price"], downcast='float')

C:\Users\junwe\AppData\Local\Temp/ipykernel_20276/1657301221.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  air_data["price"]=  air_data['price'].str.replace(',', '').str.replace('$', '')


In [9]:
air_num = pd.DataFrame(air_data[['num_of_amenities','num_of_veri','price','number_of_reviews','beds','accommodates']])

#Q1 = air_num.quantile(0.25)
#Q3 = air_num.quantile(0.75)
#IQR = Q3 - Q1

#air_num = air_num[~((air_num < (Q1 - 1.5 * IQR)) |(air_num> (Q3 + 1.5 * IQR))).any(axis=1)]

In [10]:

air_num.head()

,num_of_amenities,num_of_veri,price,number_of_reviews,beds,accommodates
0,7,9,81.0,1,1.0,1
1,12,8,80.0,18,1.0,2
2,8,9,67.0,20,1.0,1
3,25,5,177.0,20,3.0,6
4,21,5,81.0,24,1.0,3


In [11]:
air_data['ratings'] = np.where(air_data['review_scores_rating'] >= 4.7, 'Good', 'Bad')
#air_data['ratings'] = air_data['review_scores_rating'].round(1) 
air_data['ratings'].head()
air_rate = air_data[['ratings']]
air_rate.head()

,ratings
0,Good
1,Bad
2,Good
3,Bad
4,Bad


In [12]:
air_cat = air_data[['property_type','neighbourhood_cleansed']]
air_cat.head()

,property_type,neighbourhood_cleansed
0,Private room in apartment,Woodlands
1,Private room in apartment,Bukit Timah
2,Private room in apartment,Woodlands
3,Private room in villa,Tampines
4,Private room in house,Tampines


In [13]:
ohe = OneHotEncoder()
ohe.fit(air_cat)
air_cat_ohe = pd.DataFrame(ohe.transform(air_cat).toarray(),columns=ohe.get_feature_names(air_cat.columns))
air_cat_ohe.head()

,property_type_Boat,property_type_Campsite,property_type_Chambre privée dans : appartement,property_type_Entire apartment,property_type_Entire bungalow,property_type_Entire chalet,property_type_Entire condominium,property_type_Entire guest suite,property_type_Entire guesthouse,property_type_Entire home/apt,...,neighbourhood_cleansed_Singapore River,neighbourhood_cleansed_Southern Islands,neighbourhood_cleansed_Sungei Kadut,neighbourhood_cleansed_Tampines,neighbourhood_cleansed_Tanglin,neighbourhood_cleansed_Toa Payoh,neighbourhood_cleansed_Tuas,neighbourhood_cleansed_Western Water Catchment,neighbourhood_cleansed_Woodlands,neighbourhood_cleansed_Yishun
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
air_cat_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 99 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   property_type_Boat                                4238 non-null   float64
 1   property_type_Campsite                            4238 non-null   float64
 2   property_type_Chambre privée dans : appartement   4238 non-null   float64
 3   property_type_Entire apartment                    4238 non-null   float64
 4   property_type_Entire bungalow                     4238 non-null   float64
 5   property_type_Entire chalet                       4238 non-null   float64
 6   property_type_Entire condominium                  4238 non-null   float64
 7   property_type_Entire guest suite                  4238 non-null   float64
 8   property_type_Entire guesthouse                   4238 non-null   float64
 9   property_type_Entir

In [15]:
air_ohe=pd.concat([air_num,air_cat_ohe,air_rate],sort=False,axis=1).reindex(index=air_num.index)
#air_ohe=pd.concat([air_cat_ohe,air_rate],sort=False,axis=1).reindex(index=air_rate.index)
air_ohe.head(n=20)

,num_of_amenities,num_of_veri,price,number_of_reviews,beds,accommodates,property_type_Boat,property_type_Campsite,property_type_Chambre privée dans : appartement,property_type_Entire apartment,...,neighbourhood_cleansed_Southern Islands,neighbourhood_cleansed_Sungei Kadut,neighbourhood_cleansed_Tampines,neighbourhood_cleansed_Tanglin,neighbourhood_cleansed_Toa Payoh,neighbourhood_cleansed_Tuas,neighbourhood_cleansed_Western Water Catchment,neighbourhood_cleansed_Woodlands,neighbourhood_cleansed_Yishun,ratings
0,7,9,81.0,1,1.0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Good
1,12,8,80.0,18,1.0,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bad
2,8,9,67.0,20,1.0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Good
3,25,5,177.0,20,3.0,6,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Bad
4,21,5,81.0,24,1.0,3,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Bad
5,17,5,81.0,48,2.0,3,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Bad
6,14,8,52.0,20,1.0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bad
7,15,8,40.0,13,0.0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bad
8,24,7,72.0,133,1.0,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bad
9,17,5,41.0,105,1.0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bad


In [16]:
air_ohe.info()
air_ohe_test = air_ohe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Columns: 106 entries, num_of_amenities to ratings
dtypes: float32(1), float64(100), int64(4), object(1)
memory usage: 3.4+ MB


In [17]:
#air_ohe.interpolate(method ='nearest', limit_direction ='forward', limit = 1)
air_ohe.dropna(inplace=True)

#air_ohe['ratings']=air_ohe['ratings'].round(1)
y = pd.DataFrame(air_ohe['ratings'])
#y.fillna(1, inplace=True)
y.head()

,ratings
0,Good
1,Bad
2,Good
3,Bad
4,Bad


In [18]:
X = pd.DataFrame(air_ohe.drop('ratings',axis=1))


In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier


In [ ]:
#import pydot
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size = 0.2)
#RandomForestC
# Pull out one tree from the forest
regressor = RandomForestClassifier(n_estimators = 2500, random_state = 42,warm_start = True)
regressor.fit(X_train, y_train)
Tree = regressor.estimators_[2]


y_test_pred = regressor.predict(X_test)

# Export the image to a dot file
from sklearn import tree
plt.figure(figsize=(50,50))
tree.plot_tree(Tree,filled=True, 
              rounded=True, 
              fontsize=14);

C:\Users\junwe\AppData\Local\Temp/ipykernel_20276/4234515928.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train, y_train)


In [ ]:
df=pd.DataFrame({'Actual':y_test.flatten(), 'Predicted':y_test_pred})
df

In [ ]:
print("Classification Accuracy Train \t:", regressor.score(X_train, y_train))
print("Classification Accuracy Test \t:", regressor.score(X_test, y_test))


In [ ]:
#plt.figure(figsize=(5, 7))


#ax = sb.distplot(y_test, hist=False, color="r", label="Actual Value")
#sb.distplot(y_test_pred, hist=False, color="b", label="Fitted Values" , ax=ax)


###plt.title('Actual vs Fitted Values for Ratings')


#plt.show()
#plt.close()

In [ ]:
'''from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_test_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_test_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))'''

In [ ]:
'''import numpy as np
from sklearn import metrics

print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_test_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_test_pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))
'''


In [ ]:
def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_test_pred)))) * 100

In [ ]:
#mean_absolute_percentage_error(y_test, y_test_pred)

In [ ]:
#df=pd.DataFrame({'Actual':y_test.flatten(), 'Predicted':y_test_pred})
#df.corr()

In [ ]:
#print('Explained Variance Score:', metrics.explained_variance_score(y_test, y_test_pred))
#print('Max Error:', metrics.max_error(y_test, y_test_pred))
#print('Mean Squared Log Error:', metrics.mean_squared_log_error(y_test, y_test_pred))
#print('Median Absolute Error:', metrics.median_absolute_error(y_test, y_test_pred))
##print('R^2:', metrics.r2_score(X_train, y_train))
#print('Mean Poisson Deviance:', metrics.mean_poisson_deviance(y_test, y_test_pred))
#print('Explained Variance Score Train:', regressor.score(X_train, y_train))
#print('Explained Variance Score Test:', regressor.score(X_test, y_test))
#print('RMSE test:', np.sqrt(mean_squared_error(y_test, y_test_pred)))


# Adjusted R^2


In [ ]:
X_test.shape
apple = 1 - ( 1-regressor.score(X_test, y_test) ) * ( len(y_test) - 1 ) / ( len(y_test) - X_test.shape[1] - 1 )
print(apple)

In [ ]:
#r2 = 1 - np.sum((y_test_pred - y_test.flatten())**2) / np.sum((y_test.flatten() - np.mean(y_test.flatten()))**2)
#N=y_test.shape[0]
#p=105
#x = (1-r2)
#y = (N-1) / (N-p-1)
#adj_rsquared = (1 - (x * y))
#print("Adjusted-R2 : " , adj_rsquared)
#print(N)

In [ ]:
'''r2_sk = metrics.r2_score(y_test.flatten(),y_test_pred)
N=y_test.shape[0]
p=105
x = (1-r2)
y = (N-1) / (N-p-1)
adj_rsquared = (1 - (x * y))
print("Adjusted-R2 : " , adj_rsquared) '''

In [ ]:
air_data

In [ ]:
air_num = pd.DataFrame(air_data[['num_of_amenities','num_of_veri','price','number_of_reviews','beds','accommodates','ratings']])
air_num.corr()

In [ ]:
'''f = plt.figure(figsize=(12, 12))
sb.heatmap(air_num.corr(), vmin = -1, vmax = 1, annot = True, fmt = ".2f")'''

In [ ]:
from sklearn.metrics import accuracy_score
cmTest = confusion_matrix(y_test, y_test_pred)
acTest = accuracy_score(y_test, y_test_pred)
print(cmTest)
print("The Test Data Accuracy is: ",acTest)
TP = cmTest[1][1]
FN = cmTest[1][0]
FP = cmTest[0][1]
TN = cmTest[0][0]
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# False negative rate
FNR = FN/(TP+FN)
print("True Positive Rate is  (Train Data): ",TPR)
print("False Positive Rate is (Train Data): ",FNR)

In [ ]:
sb.heatmap(data=confusion_matrix(y_test, y_test_pred) ,annot =True, fmt=".0f", annot_kws={"size": 18})
plt.show

In [ ]:
y_train_pred = regressor.predict(X_train)

cmTrain = confusion_matrix(y_train, y_train_pred)
acTrain = accuracy_score(y_train, y_train_pred)
print(cmTrain)
print(acTrain)
TP = cmTrain[1][1]
FN = cmTrain[1][0]
FP = cmTrain[0][1]
TN = cmTrain[0][0]
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# False negative rate
FNR = FN/(TP+FN)
print("True Positive Rate is  (Train Data): ",TPR)
print("False Positive Rate is (Train Data): ",FNR)

In [ ]:
sb.heatmap(data=confusion_matrix(y_train, y_train_pred) ,annot =True, fmt=".0f", annot_kws={"size": 18})
plt.show